# 🍽️ SISTEM PREDIKSI MAKANAN DAN ESTIMASI NILAI GIZI BERBASIS CITRA MENGGUNAKAN DEEP LEARNING
This notebook simulates a Flask-based food image classifier and nutrition predictor.

In [46]:
import os
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, transforms
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
import ipywidgets as widgets


## 📁 Sel 2: Definisikan Path dan Load Dataset Nutrisi


In [47]:
# Direktori utama (ubah sesuai lokasi file)
BASE_DIR = '.'

# Lokasi model dan data nutrisi
MODEL_PATH = os.path.join(BASE_DIR, 'models', 'food101_model.pth')
NUTRITION_PATH = os.path.join(BASE_DIR, 'data', 'nutrition.csv')
IMAGE_PATH = os.path.join(BASE_DIR, 'static', 'images')

# Load data nutrisi ke DataFrame
nutrition_df = pd.read_csv(NUTRITION_PATH)
nutrition_df.head()


,label,weight,calories,protein,carbohydrates,fats,fiber,sugars,sodium
0,apple_pie,80,240,2,36,10,2,16,120
1,apple_pie,100,300,3,45,12,2,20,150
2,apple_pie,120,360,4,54,14,3,24,180
3,apple_pie,150,450,5,68,18,3,30,225
4,apple_pie,200,600,6,90,24,4,40,300


## 🧠 Sel 3: Load Model Food101 ResNet50

In [48]:
def load_model():
    model = models.resnet50()
    model.fc = nn.Sequential(
        nn.Dropout(0.4),
        nn.Linear(model.fc.in_features, 101)
    )
    checkpoint = torch.load(MODEL_PATH, map_location='cpu')
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    idx_to_class = checkpoint['idx_to_class']
    return model, idx_to_class

model, idx_to_class = load_model()
print(f"Model dimuat dengan {len(idx_to_class)} kelas.")


Model dimuat dengan 101 kelas.


## 🖼️ Sel 4: Fungsi Preprocessing Gambar

In [49]:
def preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
    image = Image.open(image_path).convert('RGB')
    tensor = transform(image).unsqueeze(0)
    return tensor, image


## 🔍 Sel 5: Fungsi Prediksi Label dan Confidence

In [50]:
def predict_label(tensor):
    with torch.no_grad():
        output = model(tensor)
        prob = F.softmax(output, dim=1)
        top_prob, top_idx = torch.max(prob, 1)
        label = idx_to_class[top_idx.item()]
        confidence = top_prob.item()
    return label, confidence


## 🥗 Sel 6: Fungsi Mendapatkan Berat Tersedia per Kategori

In [51]:
def get_weight_options(categories):
    weight_options = {}
    for cat in categories:
        weights = nutrition_df[nutrition_df['label'].str.contains(cat, case=False)]['weight'].unique()
        weight_options[cat] = sorted(weights.tolist())
    return weight_options


## 🔁 Sel 7: Fungsi Mendapatkan Data Nutrisi Berdasarkan Label dan Berat

In [52]:
def get_nutrition(label, weight):
    filtered = nutrition_df[nutrition_df['label'].str.contains(label, case=False)]
    filtered = filtered[filtered['weight'] == float(weight)]
    if filtered.empty:
        return None
    row = filtered.iloc[0]
    return {
        'Calories': row['calories'],
        'Protein (g)': row['protein'],
        'Carbohydrates (g)': row['carbohydrates'],
        'Fat (g)': row['fats'],
        'Fiber (g)': row['fiber'],
        'Sugars (g)': row['sugars'],
        'Sodium (mg)': row['sodium']
    }


## ▶️ Sel 8: Siapkan Widget Interaktif untuk Pilihan Kategori dan Berat

In [53]:
# Dapatkan daftar kategori berdasarkan folder gambar
categories = [cat for cat in os.listdir(IMAGE_PATH) if os.path.isdir(os.path.join(IMAGE_PATH, cat))]
weight_options = get_weight_options(categories)

category_dropdown = widgets.Dropdown(options=categories, description='Kategori:')
weight_dropdown = widgets.Dropdown(description='Berat (g):')

def update_weight_options(change=None):
    selected_cat = category_dropdown.value
    weights = weight_options.get(selected_cat, [100])
    weight_dropdown.options = weights

category_dropdown.observe(update_weight_options, names='value')
update_weight_options()


## 9. Fungsi dan Tombol Prediksi

In [56]:
predict_button = widgets.Button(description="Prediksi")
output = widgets.Output()

def on_predict_clicked(b):
    with output:
        output.clear_output()
        category = category_dropdown.value
        weight = float(weight_dropdown.value)
        
        image_dir = os.path.join(IMAGE_PATH, category)
        image_files = [f for f in os.listdir(image_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        if not image_files:
            print("Tidak ada gambar dalam kategori ini.")
            return
        
        image_name = random.choice(image_files)
        image_path = os.path.join(image_dir, image_name)
        tensor, image = preprocess_image(image_path)
        label, confidence = predict_label(tensor)
        nutrition = get_nutrition(label, weight)
        
        print(f"📷 Gambar: {image_name}")
        print(f"🍽️ Prediksi: {label}")
        print(f"🎯 Akurasi: {confidence*100:.2f}%")
        print("📊 Nutrisi:")
        if nutrition:
            for k, v in nutrition.items():
                print(f"  {k}: {v}")
        else:
            print("  Data nutrisi tidak ditemukan untuk pilihan ini.")
        
        plt.imshow(image)
        plt.axis('off')
        plt.title(f"Prediksi: {label} ({confidence*100:.1f}%)")
        plt.show()

predict_button.on_click(on_predict_clicked)


## 10. Tampilkan Widget Interaktif

In [57]:
display(category_dropdown, weight_dropdown, predict_button, output)


Dropdown(description='Kategori:', options=('apple_pie', 'baby_back_ribs', 'baklava', 'beef_carpaccio', 'beef_t…

Dropdown(description='Berat (g):', options=(80, 100, 120, 150, 200), value=None)

Button(description='Prediksi', style=ButtonStyle())

Output()